In [44]:
import requests

url = "http://api.weatherapi.com/v1/current.json" # 현재 날씨를 위한 요청

parameters = {
    "key": WEATHER_API_KEY, # API 키 지정
    "q": "Seoul", # 서울의 날씨
}

r = requests.get(url, params=parameters)

In [27]:
import requests
import os

WEATHER_API_KEY = os.environ["WEATHER_API_KEY"]
city = "Seoul" # 도시는 서울로 지정
lang_code = "ko"

url = "http://api.weatherapi.com/v1/current.json" # 현재 날씨를 위한 요청

parameters = {
    "key": WEATHER_API_KEY, # API 키 지정
    "q": city,
    "lang": lang_code
}
r = requests.get(url, params=parameters)
current_weather = r.json()
print(current_weather)

{'location': {'name': 'Seoul', 'region': '', 'country': 'South Korea', 'lat': 37.57, 'lon': 127.0, 'tz_id': 'Asia/Seoul', 'localtime_epoch': 1710265233, 'localtime': '2024-03-13 2:40'}, 'current': {'last_updated_epoch': 1710264600, 'last_updated': '2024-03-13 02:30', 'temp_c': 2.0, 'temp_f': 35.6, 'is_day': 0, 'condition': {'text': '화창함', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 0, 'feelslike_c': 0.4, 'feelslike_f': 32.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 6.0, 'gust_kph': 9.6}}


In [28]:
name = current_weather['location']['name']
localtime = current_weather['location']['localtime']
temp_c = current_weather['current']['temp_c']
temp_f = current_weather['current']['temp_f']
condition = current_weather['current']['condition']['text']

print(f"현재({localtime}) {name}의 날씨 정보: {condition}, 섭씨 {temp_c}도, 화씨 {temp_f}도")

현재(2024-03-13 2:40) Seoul의 날씨 정보: 화창함, 섭씨 2.0도, 화씨 35.6도


In [39]:
import requests
import os
import json
import re

def get_current_weather(location, unit="섭씨"):
    
    reg = re.compile(r'[a-zA-Z]') # 영어 입력인지를 검사하는 정규식
    
    if reg.match(location): # 영어로 도시 이름을 지정한 경우
        city = location # 영어 도시 이름을 바로 지정
    else: # 영어로 지정하지 않은 경우
        city_names = {"서울": "Seoul", "인천": "Incheon", "대전": "Daejeon",
                      "대구": "Daegu", "부산": "Busan", "광주": "Gwangju",
                      "수원": "Suwon", "파리": "Paris", "뉴욕": "New York"}
        city = city_names[location] # 한글 도시 이름을 영어로 변경
        
    WEATHER_API_KEY = os.environ["WEATHER_API_KEY"] # API 키 지정
    
    url = "http://api.weatherapi.com/v1/current.json"
    parameters = {"key":WEATHER_API_KEY, "q":city}
    
    r = requests.get(url, params=parameters)
    current_weather = r.json()
    
    name = current_weather['location']['name'] # 설정 지역
    localtime = current_weather['location']['localtime'] # 날짜 및 시각
    temp_c = current_weather['current']['temp_c'] # 섭씨온도
    temp_f = current_weather['current']['temp_f'] # 화씨온도
    condition_text = current_weather['current']['condition']['text'] # 날씨 상태
    
    # unit 지정에 따라서 섭씨온도 혹은 화씨온도를 지정
    if unit == "섭씨":
        temp = temp_c
    elif unit == "화씨":
        temp = temp_f
    else:
        unit == "섭씨"
        temp = temp_c
        
        
    weather_info = {
        "location": name,
        "temperature": temp,
        "unit": unit,
        "current weather": condition_text,
        "local time": localtime
    }
    
    return json.dumps(weather_info, ensure_ascii=False) # JSON 형식으로 반환

In [30]:
get_current_weather('서울')

'{"location": "Seoul", "temperature": 2.0, "unit": "섭씨", "current weather": "Clear", "local time": "2024-03-13 2:40"}'

In [31]:
get_current_weather('파리')

'{"location": "Paris", "temperature": 11.0, "unit": "섭씨", "current weather": "Moderate rain", "local time": "2024-03-12 18:42"}'

In [32]:
import openai
import os

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

# Chat Completion API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
def run_conversation(user_query):
    # 사용자 입력
    messages = [{"role":"user", "content":user_query}]
    
    # 함수 정보 입력
    functions = [
        {
            "name":"get_current_weather",
            "description": "도시 이름을 입력해 현재 날씨, 날짜, 시각, 몇 시인지 가져오기",
            "parameters": {
                "type": "object",
                "properties":{
                    "location": {
                        "type": "string",
                        "description": "도시 이름, 예를 들면, 서울, 부산, 대전",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["섭씨", "화씨"],
                        "description": "온도 단위로 섭씨 혹은 화씨",
                    },
                },
                "required": ["location"], # 필수 입력 변수 지정
            }
        }
    ]
    
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
        # model="gpt-3.5-turbo"
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto"
    )
    
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 모델의 응답 메시지
    
    if response_message.get("function_call"): # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # (주의: JSON 응답이 항상 유효하지 않을 수 있음)
        
        # 호출할 함수 이름을 지정
        # (아래는 하나의 함수를 지정했지만 여러 함수 지정 가능)
        available_functions = {"get_current_weather": get_current_weather}
        
        # 함수 이름 추출
        function_name = response_message["function_call"]["name"]
        
        # 호출할 함수 선택
        function_to_call = available_functions[function_name]
        
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # 함수 호출 및 반환 결과 받기
        function_response = function_to_call(
            location=function_args.get("location"), # 인수 지정
            unit=function_args.get("unit")
        )
        print("[호출한 함수의 응답 결과]\n", function_response)
        
        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고,
        #        Chat Completions API 모델로 보내 응답받기
        
        # 함수 호출 결과를 기존 메시지에 추가히기
        messages.append(response_message) # 기존 messages에 조력자 응답 추가
        messages.append(                  # 함수와 함수 호출 결과 추가
            {
                "role": "function",       # roll: function으로 지정 
                "name": function_name,    # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.ChatCompletion.create(
            # model="get-3.5-turbo",
            model="gpt-3.5-turbo",
            messages=messages,
        )
        return second_response # 두 번째 응답 반환
    
    return response_message # 응답 메시지 반환

In [33]:
user_query = "현재 수원의 날씨는 어떠한가요?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n",response_content)

[호출한 함수의 응답 결과]
 {"location": "Suwon", "temperature": 1.0, "unit": null, "current weather": "Partly cloudy", "local time": "2024-03-13 2:40"}
[최종 응답 결과]
 수원의 현재 날씨는 구름이 조금 끼여 있고 온도는 1도입니다.


In [34]:
json.dumps(response, ensure_ascii=False)

'{"id": "chatcmpl-920QqanQIbfdY4xPn1TTMgzsdDLKi", "object": "chat.completion", "created": 1710265344, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "수원의 현재 날씨는 구름이 조금 끼여 있고 온도는 1도입니다."}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 97, "completion_tokens": 32, "total_tokens": 129}, "system_fingerprint": "fp_4f0b692a78"}'

In [35]:
user_query = "지금 뉴욕의 날씨는?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[호출한 함수의 응답 결과]
 {"location": "New York", "temperature": 12.8, "unit": null, "current weather": "Partly cloudy", "local time": "2024-03-12 13:40"}
[최종 응답 결과]
 지금 뉴욕은 구름 많은 날씨이고 기온은 12.8°C 입니다. 현재 현지 시간은 오후 1시 40분입니다.


In [40]:
user_query = " 현재 뉴욕의 날씨는? 온도는 화씨 표시"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[호출한 함수의 응답 결과]
 {"location": "New York", "temperature": 55.0, "unit": "화씨", "current weather": "Partly cloudy", "local time": "2024-03-12 13:43"}
[최종 응답 결과]
 현재 뉴욕의 날씨는 일부 구름이 끼어있으며, 기온은 55°F 입니다. 현재 현지 시각은 2024-03-12 13:43 입니다.


In [41]:
user_query = " 서울의 현재 시각은?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[호출한 함수의 응답 결과]
 {"location": "Seoul", "temperature": 1.0, "unit": null, "current weather": "Clear", "local time": "2024-03-13 2:46"}
[최종 응답 결과]
 서울의 현재 시각은 2024-03-13 2:46이며, 날씨는 맑고 온도는 1도입니다.


In [43]:
user_query = " 파리는 지금 몇 시야?"
response = run_conversation(user_query)
response_content = response["choices"][0]["message"]["content"]

print("[최종 응답 결과]\n", response_content)

[호출한 함수의 응답 결과]
 {"location": "Paris", "temperature": 11.0, "unit": null, "current weather": "Moderate rain", "local time": "2024-03-12 18:46"}
[최종 응답 결과]
 파리는 현재 18:46 시입니다. (지역 시간 기준) 현재 기온은 11도이며, 비가 조금 오고 있습니다.
